In [11]:
import os
import re
import pymatgen as mg
import pymatgen.analysis.diffraction as anadi
import pymatgen.analysis.diffraction.xrd as xrd
import numpy as np
import glob
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
import time

In [12]:
torch.set_default_dtype(torch.float64)

torch.set_printoptions(precision=8)

patt_xrd = xrd.XRDCalculator('CuKa')

train_path='/home/mii/Desktop/cpx321/train/'

test_path='/home/mii/Desktop/cpx321/test/'

global sample_num, rmat_num, series_num
sample_num=1 #output of G
rmat_num=28  #row nums of the matrix for the input of CNN 
#series_num=3 #input of D


In [21]:
def get_energy(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    energy=round(np.float64(float(energy_string)),5)
    return energy

def linear_transform(energy):
    global extend_num, move_num
    energy_transform=(energy-move_num)*extend_num
    return energy_transform
def inverse_transform(energy_transform):
    global extend_num, move_num
    energy=energy_transform/extend_num+move_num
    return energy
def get_energy_per_atom(energy):
    energy_per_atom=energy/atoms_num
    return energy_per_atom

In [24]:
global extend_num, move_num

In [25]:
extend_num=1000

In [23]:
move_num=get_energy(train_path+'01666/')
print(move_num)

-91.62666


In [26]:
def random_xxpsk(file_path):
    folder=np.random.choice(glob.glob(file_path +"*"))
    #pos_name=folder+'/POSCAR'
    #out_name=folder+'/OUTCAR'
    return folder

def tomgStructure(folder):
    POSfile=folder+'/POSCAR'      
    R_mgS=mg.Structure.from_file(POSfile)
    return R_mgS

###
##input_data_to_model
###
def get_xrdmat3(mgStructure):
    global rmat_num
    xrd_data3 =patt_xrd.get_pattern(mgStructure)
    i_column = rmat_num
    xrow = len(xrd_data3.x)
    xrd_x= []
    xrd_y= []
    xxx=[]
    yyy=[]
    xrd_mat3=[]
    if xrow < i_column:
        #try:
         #   xrd_data3.y >1
        #except:            
        for i in xrd_data3.x:
            xxx.append(i)
        for j in xrd_data3.y:
            yyy.append(j)
        for i in range(0,i_column-xrow):
            xxx.append(0)
            yyy.append(0)
        xrd_x=np.asarray(xxx)
        xrd_y=np.asarray(yyy)
    if xrow > i_column:
        xrd_x=xrd_data3.x[:i_column]
        xrd_y=xrd_data3.y[:i_column]
    if xrow == i_column:
        xrd_x= xrd_data3.x
        xrd_y= xrd_data3.y
    xrd_x=np.sin(np.dot(1/180*np.pi,xrd_x))
    xrd_y=np.dot(1,xrd_y)
    xrd_mat3.append(xrd_x)
    xrd_mat3.append(xrd_y)
    xrd_mat3=np.array(xrd_mat3)
    return xrd_mat3

   
###
##input_data_as_knowlegde
###
'''
def get_Gibbs(folder):
    energy_string=os.popen('grep TOTEN '+folder+'/OUTCAR | tail -1').read().split(' ')[-2]
    Gibbs=np.float64(float(energy_string))
    Gibbs=round(Gibbs,6)
    return Gibbs
'''
##
###
def get_atoms_num(folder2):
    xxx=tomgStructure(folder2)
    anum=len(xxx.sites)
    return anum


###
##input_data_for_G
###
def GANs_Gmat(Random_Structure):
    global rmat_num
    RS_xrdmat = get_xrdmat3(Random_Structure)
    multimat3_RS =  np.zeros((rmat_num,rmat_num),dtype='float32')
    multimat3_RS = np.asarray((np.dot(RS_xrdmat.T, RS_xrdmat)))
    return multimat3_RS

In [27]:
class GCNNet(nn.Module):
    def __init__(self, input_size=(sample_num,28,28)):
        super(GCNNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(#(3,28,28)
                in_channels=sample_num,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(32,28,28)
            nn.ReLU(),#->(32,28,28)
            nn.MaxPool2d(kernel_size=2),
        )#->(#->(32,14,14))
        self.conv2=nn.Sequential(#->(32,14,14))
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=5,
                stride=1,
                padding=2,
            ),#->(64,14,14)
            nn.ReLU(),#->(64,14,14)
            nn.MaxPool2d(kernel_size=2),#->(64,7,7)
        )
        self.out=nn.Sequential(
            nn.Linear(64*7*7,128),
            nn.ReLU(),
            nn.Linear(128,sample_num),            
        )
        
    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x) #batch(64,7,7)
        x=x.view(x.size(0),-1) #(batch, 64*7*7)
        output=torch.unsqueeze(self.out(x),dim=0)
        return output

In [28]:
G1= GCNNet()
opt_G1=torch.optim.Adam(G1.parameters(),lr=0.0001)
#mat_Gl=[]    
#pre_gg=[]

In [ ]:
file_path=train_path
tfset=[]
for step in range(1,11):       
 
    sample_path=[]
    for i in range(1,sample_num + 1):
        path_ = random_xxpsk(file_path)
        sample_path.append(path_)
        tfset.append(path_)
    E_Gibbs=[]
    for path1_ in sample_path:
        try:
            total_energy=get_energy(path1_)
            tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
        except:
            print(path1_)
            
        E_Gibbs.append(tt_energy)
        
    E_Gibbs=np.asarray(E_Gibbs,dtype=np.float64)       
    E_Gibbs=Variable(torch.from_numpy(E_Gibbs[np.newaxis,np.newaxis,:]),requires_grad=True)
    #print(tfactor.shape)
    
    g_in=[]
    for path2_ in sample_path:
        path2_=str(path2_)                
        
        try:
            tomgS=tomgStructure(path2_)
            #print(tomgS)
            gin=GANs_Gmat(tomgS)
            #print(gin)
        except:
            pass
        g_in.append(gin)
       
    g_in=np.asarray(g_in)
    g_in=g_in[np.newaxis,:,:,:] 
    g_in=np.asarray(g_in,dtype=np.float64) 
    g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
    Gout=G1(g_in)    
    
    G1_loss=torch.abs(torch.mean(Gout-float(tt_energy)))
    

    opt_G1.zero_grad()
    G1_loss.backward()
    opt_G1.step()




    
    
    if step%10==0:
        print('----------------------------------test-------------------------')
        file_path_test=test_path
        E_Gibbs=[]
        E_Gmodel=[]
        for step_test in range(10):
            sample_path=[]
            for i in range(1,sample_num + 1):
                path_ = random_xxpsk(file_path_test)
                sample_path.append(path_)
            
            for path1_ in sample_path:
                try:
                    total_energy=get_energy(path1_)
                    #tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
                except:
                    print(path1_)
                    
            tt_energy=float(total_energy)
            print('DFT',tt_energy)
            E_Gibbs.append(tt_energy)
        
                   

        #print(tfactor.shape)
    
            g_in=[]
            for path2_ in sample_path:
                path2_=str(path2_)                
        
                try:
                    tomgS=tomgStructure(path2_)
            #print(tomgS)
                    gin=GANs_Gmat(tomgS)
            #print(gin)
                except:
                    pass
                g_in.append(gin)
       
            g_in=np.asarray(g_in)
            g_in=g_in[np.newaxis,:,:,:] 
            g_in=np.asarray(g_in,dtype=np.float64) 
            g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
            Gout=G1(g_in)
        #print(Gout.shape)
    
            G_data=Gout.data.numpy().mean()
            G_data=inverse_transform(G_data)
            print('G_predict',G_data)
            E_Gmodel.append(G_data)

        
            
            
            print('error',abs(tt_energy-G_data))
            
        print('------------------end-test----------------------------')
        print(abs(np.asarray(E_Gibbs)-np.asarray(E_Gmodel)).mean())
        X=np.asarray(E_Gibbs)
        Y=np.asarray(E_Gmodel)

        xbar=X.mean()
        ybar=Y.mean()
        SSR=0
        varX=0
        varY=0
        for i in range(len(X)):
            diffxxbar=X[i]-xbar
            diffyybar=Y[i]-ybar
            SSR+=(diffxxbar*diffyybar)
            varX+=diffxxbar**2
            varY+=diffyybar**2
    
        SST=math.sqrt(varX+varY)
        R2=(SSR/SST)**2
        print("R2:",R2)
        r2.append(R2)
    else:
        pass
    
            


In [29]:
trainfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        trainfile.append(ieach)
i=0
loss_set=[]
for path_ in trainfile:
    E_Gibbs=[]
    try:
        total_energy=get_energy(path_)
        tt_energy=linear_transform(total_energy)
            #print(samp_Gibbs)
    except:
        print(path_)
            
    E_Gibbs.append(tt_energy)
        
    E_Gibbs=np.asarray(E_Gibbs,dtype=np.float64)       
    E_Gibbs=Variable(torch.from_numpy(E_Gibbs[np.newaxis,np.newaxis,:]),requires_grad=True)
    
    g_in=[]
           
     
    try:
        tomgS=tomgStructure(path_)
            #print(tomgS)
        gin=GANs_Gmat(tomgS)
            #print(gin)
    except:
        pass
    g_in.append(gin)
       
    g_in=np.asarray(g_in)
    g_in=g_in[np.newaxis,:,:,:] 
    g_in=np.asarray(g_in,dtype=np.float64) 
    g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
    
    Gout=G1(g_in)    
    
    G1_loss=torch.abs(torch.mean(Gout-float(tt_energy)))
    

    opt_G1.zero_grad()
    G1_loss.backward()
    opt_G1.step()
    
    i += 1
    loss_set.append(G1_loss)
    print(i,": ",G1_loss)

1 :  tensor(108.79977454, grad_fn=<AbsBackward>)
2 :  tensor(35.57115951, grad_fn=<AbsBackward>)


/opt/anaconda/envs/py36ML/lib/python3.6/site-packages/pymatgen/io/vasp/inputs.py:390: UserWarning: Elements in POSCAR cannot be determined. Defaulting to false names H H H H H H He He He He He He Li Li Li Li Li Li Li Li Li Li Li Li Be Be Be Be Be Be.
  " ".join(atomic_symbols))


3 :  tensor(76.44226283, grad_fn=<AbsBackward>)
4 :  tensor(39.93991404, grad_fn=<AbsBackward>)
5 :  tensor(163.56482047, grad_fn=<AbsBackward>)
6 :  tensor(144.69229467, grad_fn=<AbsBackward>)
7 :  tensor(53.81307115, grad_fn=<AbsBackward>)
8 :  tensor(78.30669853, grad_fn=<AbsBackward>)
9 :  tensor(26.27916994, grad_fn=<AbsBackward>)
10 :  tensor(36.05518601, grad_fn=<AbsBackward>)
11 :  tensor(76.26535264, grad_fn=<AbsBackward>)
12 :  tensor(15.56447121, grad_fn=<AbsBackward>)
13 :  tensor(139.10705518, grad_fn=<AbsBackward>)
14 :  tensor(11.98585867, grad_fn=<AbsBackward>)
15 :  tensor(90.16460847, grad_fn=<AbsBackward>)
16 :  tensor(11.48699546, grad_fn=<AbsBackward>)
17 :  tensor(16.82284977, grad_fn=<AbsBackward>)
18 :  tensor(8.19886596, grad_fn=<AbsBackward>)
19 :  tensor(25.35646307, grad_fn=<AbsBackward>)
20 :  tensor(115.65033812, grad_fn=<AbsBackward>)
21 :  tensor(33.17592409, grad_fn=<AbsBackward>)
22 :  tensor(45.54943075, grad_fn=<AbsBackward>)
23 :  tensor(12.89243357

170 :  tensor(30.99371262, grad_fn=<AbsBackward>)
171 :  tensor(76.64102872, grad_fn=<AbsBackward>)
172 :  tensor(42.39747769, grad_fn=<AbsBackward>)
173 :  tensor(30.31207608, grad_fn=<AbsBackward>)
174 :  tensor(10.31425313, grad_fn=<AbsBackward>)
175 :  tensor(7.48069817, grad_fn=<AbsBackward>)
176 :  tensor(20.00666812, grad_fn=<AbsBackward>)
177 :  tensor(26.48470430, grad_fn=<AbsBackward>)
178 :  tensor(2.93747963, grad_fn=<AbsBackward>)
179 :  tensor(13.86081812, grad_fn=<AbsBackward>)
180 :  tensor(4.91426116, grad_fn=<AbsBackward>)
181 :  tensor(34.99410258, grad_fn=<AbsBackward>)
182 :  tensor(7.87527351, grad_fn=<AbsBackward>)
183 :  tensor(2.10358701, grad_fn=<AbsBackward>)
184 :  tensor(32.68468174, grad_fn=<AbsBackward>)
185 :  tensor(7.63386920, grad_fn=<AbsBackward>)
186 :  tensor(53.68946397, grad_fn=<AbsBackward>)
187 :  tensor(21.63530693, grad_fn=<AbsBackward>)
188 :  tensor(13.61275078, grad_fn=<AbsBackward>)
189 :  tensor(4.90166137, grad_fn=<AbsBackward>)
190 :  

336 :  tensor(10.43201847, grad_fn=<AbsBackward>)
337 :  tensor(14.72785412, grad_fn=<AbsBackward>)
338 :  tensor(7.78017366, grad_fn=<AbsBackward>)
339 :  tensor(7.99449230, grad_fn=<AbsBackward>)
340 :  tensor(7.76338454, grad_fn=<AbsBackward>)
341 :  tensor(12.68392407, grad_fn=<AbsBackward>)
342 :  tensor(5.19514958, grad_fn=<AbsBackward>)
343 :  tensor(3.74151317, grad_fn=<AbsBackward>)
344 :  tensor(3.43525630, grad_fn=<AbsBackward>)
345 :  tensor(23.97513552, grad_fn=<AbsBackward>)
346 :  tensor(13.19227400, grad_fn=<AbsBackward>)
347 :  tensor(0.29154891, grad_fn=<AbsBackward>)
348 :  tensor(16.55237799, grad_fn=<AbsBackward>)
349 :  tensor(0.06961092, grad_fn=<AbsBackward>)
350 :  tensor(2.32763368, grad_fn=<AbsBackward>)
351 :  tensor(8.39424821, grad_fn=<AbsBackward>)
352 :  tensor(6.89915392, grad_fn=<AbsBackward>)
353 :  tensor(14.13308909, grad_fn=<AbsBackward>)
354 :  tensor(1.74569342, grad_fn=<AbsBackward>)
355 :  tensor(0.70639947, grad_fn=<AbsBackward>)
356 :  tensor

In [38]:
torch.save(G1.state_dict(),"/home/mii/Desktop/cpx321/G1_GCN-321-wo.pkl") 

In [31]:

E_Gibbs_test=[]
E_Gmodel_test=[]
abserrset=[]
MSEset=[]
err0set=[]
testfile=[]
for m1,n1,fname in os.walk(test_path):
    for ieach in n1:
        ieach=test_path+ieach
        testfile.append(ieach)
start=time.time()        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        #GGG=inverse_transform(GGG)
        E_Gibbs_test.append(GGG)
        
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        #G_data=get_energy_per_atom(G_data)
        E_Gmodel_test.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        mse=(G_data-GGG)**2
        abserrset.append(abserr)
        MSEset.append(mse)
        err0=abs(abserr/GGG)
        err0set.append(err0)
    except:
        print(path_)
end=time.time()
print(end-start)

232.86536717414856


In [32]:
232.86536717414856

232.86536717414856

In [33]:
print(np.asarray(abserrset).mean())

print(np.asarray(MSEset).mean())

0.006685158886799968
7.339875966660048e-05


In [34]:
print(np.sqrt(np.asarray(MSEset).mean()))

0.008567307608963301


In [35]:
E_Gibbs_t=[]
E_Gmodel_t=[]
abs_t_errset=[]
err_t_0set=[]
tMSEset=[]
testfile=[]
for m1,n1,fname in os.walk(train_path):
    for ieach in n1:
        ieach=train_path+ieach
        testfile.append(ieach)
#'''        
for path_ in testfile:
    try:
        GGG=get_energy(path_)
        
        #GGG=get_energy_per_atom(GGG)
        E_Gibbs_t.append(GGG)
        g_in=[]
        tomgS=tomgStructure(path_)
        gin=GANs_Gmat(tomgS)
        g_in.append(gin)
        g_in=np.asarray(g_in)
        g_in=g_in[np.newaxis,:,:,:]
        g_in=np.asarray(g_in,dtype=np.float64)
        g_in=Variable(torch.from_numpy(g_in),requires_grad=True)
        Gout=G1(g_in)
        G_data=Gout.data.numpy().mean()
        G_data=inverse_transform(G_data)
        #G_data=get_energy_per_atom(G_data)
        E_Gmodel_t.append(G_data)
        #print(G_data)
        #print(GGG)
        abserr=abs(G_data-GGG)
        tmse=(G_data-GGG)**2
        tMSEset.append(tmse)
        abs_t_errset.append(abserr)
        err0=abs(abserr/GGG)
        err_t_0set.append(err0)
    except:
        print(path_)

In [36]:
print(np.asarray(abs_t_errset).mean())

print(np.asarray(tMSEset).mean())

0.006509895514399147
7.233362479324285e-05


In [37]:
np.sqrt(7.233362479324285e-05)

0.008504917682919855